# Data setting

In [1]:
%%bash

# download ISIC sample file

function gdrive_download () {
  CONFIRM=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate "https://docs.google.com/uc?export=download&id=$1" -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')
  wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$CONFIRM&id=$1" -O $2
  rm -rf /tmp/cookies.txt
}

gdrive_download 1rDdsupDmNVfABsn2xPsPK3pCmdM_OtnX ISIC_sample.zip

sed: illegal option -- r
usage: sed script [-Ealn] [-i extension] [file ...]
       sed [-Ealn] [-i extension] [-e script] ... [-f script_file] ... [file ...]
--2020-05-19 22:18:33--  https://docs.google.com/uc?export=download&confirm=&id=1rDdsupDmNVfABsn2xPsPK3pCmdM_OtnX
Resolving docs.google.com (docs.google.com)... 216.58.197.174
Connecting to docs.google.com (docs.google.com)|216.58.197.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘ISIC_sample.zip’

     0K ...                                                    19.1M=0s

2020-05-19 22:18:34 (19.1 MB/s) - ‘ISIC_sample.zip’ saved [3268]



In [5]:
!unzip ISIC_sample.zip -d datasets

Archive:  sample_data.zip
   creating: datasets/sample_data/
  inflating: datasets/sample_data/ISIC_0000025.jpg  
  inflating: datasets/sample_data/ISIC_0000031.jpg  
  inflating: datasets/sample_data/ISIC_0000019.jpg  
  inflating: datasets/sample_data/ISIC_0000104_segmentation.png  
  inflating: datasets/sample_data/ISIC_0000003_segmentation.png  
  inflating: datasets/sample_data/ISIC_0000058_segmentation.png  
  inflating: datasets/sample_data/ISIC_0000066_segmentation.png  
  inflating: datasets/sample_data/ISIC_0000080_segmentation.png  
  inflating: datasets/sample_data/ISIC_0000047_segmentation.png  
  inflating: datasets/sample_data/ISIC_0000079_segmentation.png  
  inflating: datasets/sample_data/ISIC_0000022_segmentation.png  
  inflating: datasets/sample_data/ISIC_0000018.jpg  
  inflating: datasets/sample_data/ISIC_0000030.jpg  
  inflating: datasets/sample_data/ISIC_0000024.jpg  
  inflating: datasets/sample_data/ISIC_0000032.jpg  
  inflating: datasets/sample_data/ISIC_0

  inflating: datasets/sample_data/ISIC_0000103.jpg  
  inflating: datasets/sample_data/ISIC_0000074_segmentation.png  
  inflating: datasets/sample_data/ISIC_0000092_segmentation.png  
  inflating: datasets/sample_data/ISIC_0000011_segmentation.png  
  inflating: datasets/sample_data/ISIC_0000088.jpg  
  inflating: datasets/sample_data/ISIC_0000077.jpg  
  inflating: datasets/sample_data/ISIC_0000063.jpg  
  inflating: datasets/sample_data/ISIC_0000004.jpg  
  inflating: datasets/sample_data/ISIC_0000088_segmentation.png  
  inflating: datasets/sample_data/ISIC_0000050_segmentation.png  
  inflating: datasets/sample_data/ISIC_0000038.jpg  
  inflating: datasets/sample_data/ISIC_0000035_segmentation.png  
  inflating: datasets/sample_data/ISIC_0000014_segmentation.png  
  inflating: datasets/sample_data/ISIC_0000039.jpg  
  inflating: datasets/sample_data/ISIC_0000097_segmentation.png  
  inflating: datasets/sample_data/ISIC_0000011.jpg  
  inflating: datasets/sample_data/ISIC_0000071_s

In [6]:
# sample_data 폴더가 보이면 완료

!ls datasets

sample_data


# build_dataset.py

In [1]:
import os
import cv2
from glob import glob
import matplotlib.pyplot as plt

INPUT_SIZE = 572
OUTPUT_DIR = "./datasets/preprocessing"


Bad key "echo "backend" on line 1 in
/Users/amore/.matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.1/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
def resize_and_save(filename, output_dir, size=572):
  """ 입력 이미지 사이즈 재조정 후 output_dir에 저장 """

  # 이미지 크기 조정
  img = cv2.imread(filename)
  img = cv2.resize(img, (size, size), cv2.INTER_LINEAR)

  # 저장할 폴더 생성
  os.makedirs(output_dir, exist_ok=True)

  output_file_path = os.path.join(output_dir, os.path.split(filename)[-1])
  cv2.imwrite(output_file_path, img)


In [3]:
img_paths = glob("./datasets/sample_data/*.jpg")
mask_paths= glob("./datasets/sample_data/*.png")
img_paths = sorted(img_paths, key=lambda x: int(os.path.split(x)[-1].split(".")[0].split("_")[-1]))
mask_paths= sorted(mask_paths, key=lambda x: int(os.path.split(x)[-1].split(".")[0].split("_")[-2]))

In [4]:
# save train images and masks
for p in img_paths[:90]:
  resize_and_save(p, OUTPUT_DIR+"/images/train", size=INPUT_SIZE)
for p in mask_paths[:90]:
  resize_and_save(p, OUTPUT_DIR+"/masks/train", size=INPUT_SIZE)

# save val images and masks
for p in img_paths[90:95]:
  resize_and_save(p, OUTPUT_DIR+"/images/val", size=INPUT_SIZE)
for p in mask_paths[90:95]:
  resize_and_save(p, OUTPUT_DIR+"/masks/val", size=INPUT_SIZE)

# save test images and masks
for p in img_paths[95:]:
  resize_and_save(p, OUTPUT_DIR+"/images/test", size=INPUT_SIZE)
for p in mask_paths[95:]:
  resize_and_save(p, OUTPUT_DIR+"/masks/test", size=INPUT_SIZE)

# my_transform.py 

In [29]:
# my_transform.py

class random_rotation_transform:
    def __init__(self, max_angle=10):
        self.max_angle = max_angle
        
    def __call__(self, x):
        image = x['image']
        mask = x['mask']
        if random.random() > 0.5:
            angle = random.randint(-self.max_angle, self.max_angle)
            image = TF.rotate(image, angle)
            mask = TF.rotate(mask, angle)
        return {'image':image, 'mask':mask}
    
class normalize:
    def __call__(self, x):
        image = x['image']
        mask = x['mask']
        image = np.array(image, dtype=np.float32)
        mask = np.array(mask, dtype=np.float32)
        image = image / 255.
        mask = mask / 255.
        
        return {'image':image, 'mask':mask}
    
class to_tensor:
    def __call__(self, x):
        image = x['image']
        mask = x['mask']
        
        image = np.array(image, dtype=np.float32).transpose((2, 0, 1))
        mask = np.array(mask, dtype=np.float32)
        
        image = torch.from_numpy(image).float()
        mask = torch.from_numpy(mask).float()
        
        return {'image':image, 'mask':mask}

# data_loader.py

In [30]:
# data_loader.py

import os
import random
import numpy as np
from PIL import Image
from glob import glob

import torch
import torch.utils.data as data
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF

class DataLoaderSegmentation(data.Dataset):
    def __init__(self, folder_path, task):
        super(DataLoaderSegmentation, self).__init__()
        self.task = task # train / val / test
        self.transforms = transforms
        self.img_files = glob(os.path.join(folder_path, "images", self.task,"*.jpg"))
        self.mask_files= glob(os.path.join(folder_path, "masks", self.task, "*.png"))

    def __getitem__(self, index):
        img_path = self.img_files[index]
        mask_path= self.mask_files[index]
        image = Image.open(img_path)
        mask  = Image.open(mask_path)

        return self.mytransforms({'image':image, 'mask':mask}, self.task)

    def __len__(self):
        return len(self.img_files)

    def mytransforms(self, x:dict, task:str):
        if task == "train":            
            transforms_pipeline = transforms.Compose([
                random_rotation_transform(),
                normalize(),
                to_tensor()
            ])
        elif task == "val":
            transforms_pipeline = transforms.Compose([
                normalize(),                
                to_tensor()
            ])
        else:
            transforms_pipeline = transforms.Compose([
                normalize(),                
                to_tensor()
            ])
            
        return transforms_pipeline(x)


# Dataset 클래스 호출 

In [31]:
train_dataset = DataLoaderSegmentation(folder_path="datasets/preprocessing", task="train")
dl = DataLoader(train_dataset)

In [32]:
for d in dl:
    print(d)
    break

{'image': tensor([[[[0.5608, 0.4667, 0.5412,  ..., 0.5373, 0.5255, 0.6157],
          [0.1451, 0.3333, 0.3647,  ..., 0.3725, 0.3686, 0.2000],
          [0.2353, 0.6706, 0.6902,  ..., 0.7569, 0.7059, 0.2549],
          ...,
          [0.2196, 0.7059, 0.6902,  ..., 0.7529, 0.7451, 0.2196],
          [0.1412, 0.3490, 0.3490,  ..., 0.3569, 0.3647, 0.1725],
          [0.5765, 0.5059, 0.5804,  ..., 0.5804, 0.5255, 0.5882]],

         [[0.6667, 0.5098, 0.5059,  ..., 0.4863, 0.5059, 0.6314],
          [0.1882, 0.3098, 0.2627,  ..., 0.2824, 0.3098, 0.1804],
          [0.1922, 0.5647, 0.5059,  ..., 0.6157, 0.6039, 0.1961],
          ...,
          [0.1882, 0.6078, 0.5333,  ..., 0.5922, 0.6549, 0.1843],
          [0.1765, 0.3255, 0.2745,  ..., 0.2667, 0.3216, 0.1804],
          [0.6431, 0.5137, 0.5451,  ..., 0.5216, 0.5098, 0.6118]],

         [[0.6314, 0.4941, 0.5176,  ..., 0.5137, 0.5216, 0.6353],
          [0.1725, 0.3176, 0.2980,  ..., 0.3216, 0.3373, 0.1961],
          [0.2078, 0.6000, 0.568